In [147]:
from typing import List
import math
import random
from tqdm import tqdm

In [148]:
class MyMLP:
  def __init__(self, npl: List[int]): # npl : neurons_per_layer (input included !)
    self.d = list(npl)
    self.L = len(npl) - 1
    self.W = []
    # self.W[l][i][j]

    for l in range(0, len(self.d)):
      self.W.append([])
      if l == 0:
        continue

      for i in range(0, self.d[l - 1] + 1):
        self.W[l].append([])
        for j in range(0, self.d[l] + 1):
          rdm_value = random.random() * 2.0 - 1.0
          self.W[l][i].append(0.0 if j == 0 else rdm_value)

    self.X = []
    self.deltas = []

    for l in range(0, len(self.d)):
      self.X.append([])
      self.deltas.append([])

      for j in range(0, self.d[l] + 1):
        self.X[l].append(1.0 if j == 0.0 else 0.0)
        self.deltas[l].append(0.0)

  def _propagate(self, inputs: List[float], is_classification: bool):
    assert(len(inputs) == self.d[0])
    for j in range(len(inputs)):
      self.X[0][j + 1] = inputs[j]

    for l in range(1, self.L + 1):
      for j in range(1, self.d[l] + 1):
        signal = 0.0
        for i in range(0, self.d[l - 1] + 1):
          signal += self.W[l][i][j] * self.X[l - 1][i]
        x = signal
        if is_classification or l != self.L:
          x = math.tanh(signal)
        self.X[l][j] = x

  def predict(self, inputs: List[float], is_classification: bool):
    self._propagate(inputs, is_classification)
    return self.X[self.L][1:]

  def train(self,
            all_samples_inputs: List[List[float]],
            all_samples_expected_outputs: List[List[float]],
            is_classification: bool,
            num_iter: int,
            alpha: float # learning rate !
            ):
    assert(len(all_samples_inputs) == len(all_samples_expected_outputs))
    for _ in tqdm(range(num_iter)):
      k = random.randint(0, len(all_samples_inputs) - 1)
      inputs_k = all_samples_inputs[k]
      expected_outputs_k = all_samples_expected_outputs[k]
      self._propagate(inputs_k, is_classification) # mise à jour des Xlj

      for j in range(1, self.d[self.L] + 1):
        self.deltas[self.L][j] = self.X[self.L][j] - expected_outputs_k[j - 1]
        if is_classification:
          self.deltas[self.L][j] *= (1.0 - self.X[self.L][j] ** 2)

      for l in reversed(range(2, self.L + 1)):
        for i in range(1, self.d[l - 1] + 1):
          total = 0.0
          for j in range(1, self.d[l] + 1):
            total += self.W[l][i][j] * self.deltas[l][j]

          total *= (1.0 - self.X[l - 1][i] ** 2)
          self.deltas[l - 1][i] = total

      for l in range(1, self.L + 1):
        for i in range(0, self.d[l - 1] + 1):
          for j in range(1, self.d[l] + 1):
            self.W[l][i][j] -= alpha * self.X[l - 1][i] * self.deltas[l][j]

In [149]:
mlp = MyMLP([2, 3, 1])
print(mlp.W)
print(mlp.X)
print(mlp.deltas)

[[], [[0.0, 0.7981894439990236, 0.5685884295410522, 0.6878590798555286], [0.0, 0.7577210771839882, 0.4255303778389341, -0.4664318384223167], [0.0, -0.32577188772254373, 0.6056094432201462, -0.07139853179550149]], [[0.0, 0.6023265389895947], [0.0, -0.2751625881922759], [0.0, 0.9958794750570799], [0.0, 0.43969114883403826]]]
[[1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0]]
[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0]]


In [150]:
xor_inputs = [
    [0.0, 0.0],
    [1.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0]
]
xor_expected_outputs = [
    [-1.0],
    [1.0],
    [1.0],
    [-1.0],
]

print("Predictions avant entrainement :")
for sample_inputs in xor_inputs:
  print(mlp.predict(sample_inputs, True))

mlp.train(xor_inputs, xor_expected_outputs, True, 1000000, 0.01)
print()
print("Predictions après entrainement :")
for sample_inputs in xor_inputs:
  print(mlp.predict(sample_inputs, True))

Predictions avant entrainement :
[0.8319421294017273]
[0.8343864843322336]
[0.9129009825240344]
[0.8749413678031982]


100%|██████████| 1000000/1000000 [00:10<00:00, 96250.78it/s]


Predictions après entrainement :
[-0.993922863047017]
[0.9923754780495251]
[0.9960249797256542]
[-0.9927994984456048]
